In [1]:
%%HTML
<style>

.rendered_html {
  font-size:1.00em;
}
.rendered_html table, .rendered_html th, .rendered_html tr, .rendered_html td {
     font-size: 100%;
}

</style>

<center><h1>Data deduplication with Python</h1></center>
<center><h2><font color='grey'>Applying modern tools to the 'classic' linkage approach.</font></h2></center>

---


Today, I'm going to talk about <b>[<font color='blue'>record linkage</font>](https://en.wikipedia.org/wiki/Record_linkage) (RL)</b> - <i>the process of matching multiple records that correspond to the same entity.</i>

> #### <font color='orange'>Jupyter Notebook:</font> [github.com/meccaLeccaHi/RL_pybay2020](https://github.com/meccaLeccaHi/RL_pybay2020)
- I'll paste the link in the group chat -- <b>`clone` it</b> and <i>follow along!</i>
>
> #### <font color='purple'>Slack:</font> [#talk-data-deduplication-with-python](https://pybay2020.slack.com/archives/C018VEY8DKK)
> - Feel free to <b>ask questions</b> after the talk!

<img align="right" src="https://i.imgur.com/wr0gqAH.png" width="300" height="300">

<center><h1><font color='green'>Hi!</font></h1></center>

- I'm [Adam](https://www.adam-p-jones.com/).
    - I'm a [data scientist](https://www.linkedin.com/in/adam-p-jones/) with an interest in complex problems in healthcare.
    - Formerly a [Neuroscientist](https://pubmed.ncbi.nlm.nih.gov/?term=adam+p+jones) at the [NIH](https://www.nih.gov/).      

Past projects:
- [Maternal data linkage](https://github.com/meccaLeccaHi/record_linkage) (2019) -- Critical Juncture
    - Effort to improve understanding of perinatal health outcomes in California.
- [Testing the impact of health workers in Mali](https://www.datakind.org/blog/tracking-patients-across-years-with-record-linkage) (2020) -- DataKind SF
    - Probabilistic record linkage across consecutive census survey years.

<a id="motivation"></a>
<font color='grey'><h1>Motivation</h1></font>

---

Everyone listening to me be conceptualized as a <i>series of records</i> that have, do, or will exist.

- We all begin with a <b>birth certificate</b>. 

<br>
<center>
    <img src="assets/birth_certs.png" width="1400" alt="birth certificates">
</center>

- We all end with a <b>death certificate</b>.

<br>
<center>
    <img src="assets/death_certs.png" width="1400" alt="death certificates">
</center>

- In between, there will be <b>medical records, school records, marriage records, bank records, arrest records,</b> etc.

<br>
<center>
    <img src="assets/in_between.png" width="1200" alt="records examples">
</center>

> <b><i>Imagine</i></b> what we could learn about ourselves by integrating all of that information, from all of those different sources, into one single, cohesive story...

> ... with consistent formatting, and no missing values.

And, if data were perfectly clean, this would all reduce to a simple [`JOIN`](https://en.wikipedia.org/wiki/Join_(SQL)) operation.

### But it's <font color='#CC0033'><i>NOT</i></font>.
- This is why we need techniques to integrate inconsistent data.
- Those techniques will be the focus of this talk.

## Quick example


<b>Public health problem</b>: Joining hospital records to death certificates.

> <i>What problems are likely to occur?</i>
> 1. People change
>     - Names, addresses, ...
> 2. People make mistakes
>     - Typos, spelling errors, nicknames, abbreviations, ...
> 3. People lie
>     - Age, weight, neighborhood, ...

<a id="application"></a>
## History and Application

#### Data matching is <i>not</i> new. 
- Well before computers, we needed to match records belonging to the same individual.
- It is not known when, exactly, record linkage first began.
- We <i>do</i> know that record-keeping, itself, goes all the way back to the beginning of written-language.

<br>
<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/3/3d/Rembrandt_-_Moses_with_the_Ten_Commandments_-_Google_Art_Project.jpg" alt="Early databases" width="300">
</center>

<center><i>Possible first attempt to join two databases (Rembrandt, 1659).</i></center>

<img align="right" src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/7f/Forward_font_awesome.svg/1024px-Forward_font_awesome.svg.png" width="100">

> - But, in the interest of time, we'll skip ahead a few millenia...

#### National census
- Governments around the world rely on census data to allocate resources appropriately.
- RL plays an important role in improving the quality and accuracy of census data.

<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/85/Seal_of_the_United_States_Census_Bureau.svg/200px-Seal_of_the_United_States_Census_Bureau.svg.png" alt="Census Bureau seal" width="250">
</center>  

> The [U.S. Census Bureau](https://www.census.gov/) has played a major role in the development of RL techniques for several decades -- developed several popular algorithms widely used in RL today.

#### Medicine and public health 
- Simply consider all of the doctors, hospitals, insurance companies, and pharmacies you've interacted with and it becomes obvious why medical records are one of the largest RL application areas.
- In addition, <b>longitudinal-matching</b> of records can provide novel insights into health outcomes.

<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/45/Seattle_physician_with_patient_1999.jpg/256px-Seattle_physician_with_patient_1999.jpg" width="325" alt="Seattle physician with patient 1999" >
</center>

> In both the U.K. and Australia longitudinal data matching has been used successfully to examine health outcomes for hundreds of thousands of individuals using anonymized medical records.
> - This has lead to <b>valuable discoveries</b> regarding disease, mortality, migration, and socio-economic outcomes.

<center>
    <img src="https://upload.wikimedia.org/wikipedia/en/7/78/DB-database-icon.png" alt="database icon" width="250">
</center>

#### Customer records
- In order to effectively target their customers, businesses need to minimize the redundancy that tends to occur as a result of changes in name, address, etc.
- This requires the <b>periodic removal of redundant records</b>, in order to maintain an accurate record of their customer base (often a main source of revenue) and reach those customers effectively.

#### Genealogy

<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/fe/900-158_Ahnentafel_Herzog_Ludwig.jpg/1920px-900-158_Ahnentafel_Herzog_Ludwig.jpg" alt="family tree" width="500">
</center>

> - <b><i>Interesting Fact:</i></b> More than 10% of men and women were named 'John' and 'Mary', respectively, in 19th-century England.

- Issues like these make tools such as RL critical for genealogical databases, which are now a billion-dollar industry.
- [FamilySearch](https://en.wikipedia.org/wiki/FamilySearch) (i.e. Church of LDS) have invested heavily in RL and have published several technical reports on the subject.


<img align="right" src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b8/Man-scratching-head.gif/247px-Man-scratching-head.gif" alt="head scratch" width="100">

<a id="why"></a>
## Why do I care?

As a society, we are producing more data than ever before. 
- In order to make use of it, we need intelligent solutions to integrate data from disparate sources.

RL lets you <b><i>do more</i></b> with your data.
- RL can be used to enhance dimensions including <b>time, breadth,</b> or <b>depth</b> (of detail). 
- As a result, these tools play an important role in both <b>data warehousing</b> <i>and</i> <b>data mining</b>.
    - We can not only improve the <i>quality</i> (and statistical power) of our data, 
    - But also <i>reveal unknown relationships</i> not contained within any single database.

<a id="objectives"></a>
### Objectives
<b>1.</b> Make you aware of the specific, unique challenges involved with RL.

<b>2.</b> Give you a snapshot of the (neo-)classic record linkage approach.

<b>3.</b> Go through a demo together in which we compare the performance of a variety of classification models that we'll use for RL.

- [Motivation](#motivation)
    - [Quick example](#example)
    - [History and Application](#application)
    - [Why do I care?](#why)
        - [Objectives](#objectives)
- [Challenges](#challenges)
    - [Missing unique identifiers](#unique)
    - [Computational complexity](#complexity)
    - [Lack of training labels](#labels)
    - [Privacy](#privacy)
- [Classic record linkage](#classic)
    - [Pre-processing](#pre)
        - [Handling missing values & outliers](#handling)
        - [Segmentation](#segmentation)
        - [Phonetic encoding](#phonetic)
    - [Indexing (blocking)](#blocking)
        - [Defining blocking keys](#keys)
    - [Comparison](#comparison)
        - [Strings](#strings)
        - [Numbers](#numbers)
        - [Time](#time)
        - [Space](#space)
    - [Classification](#classification)
        - [Assignment](#assignment)
    - [Evaluation](#evaluation)
- [Demo](#demo)
    - [Comparing classifiers](#comparing)
        - [Supervised classifiers](#supervised)
        - [Unsupervised classifiers](#unsupervised)
- [Conclusions](#conclusions)
    - [Resources](#resources)


<a id="challenges"></a>
<font color='grey'><h1>Challenges</h1></font>

---

i.e. <i>'Why can't I just `JOIN`?'</i>

<center>
    <img src="assets/bear.png" alt="Why??" width="200">
</center>

<a id="unique"></a>
## Missing unique identifiers

In all of these cases, the challenge that we have to overcome is missing a unique identifier for the entities we are matching.
- <i>For example</i>, if we had perfectly accurate social security numbers for each record, the task would be reduced to a straight-forward join of two databases.

This is <i>often</i> not the case for multiple reasons:
1. Accurate record keeping is <i>hard</i>.
    - Mistakes and inaccuracies lead to undesired variation.
    
2. Privacy is usually a concern 
    - In some countries use of such identifiers is illegal.

As such, in order to match records across databases, we must use common attributes shared by both databases.
- e.g. Name, address, phone number, age.
- But, the quality of data points such as these are <i>notoriously low</i> for reasons described earlier.

<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/User-privacy-icon.svg/500px-User-privacy-icon.svg.png" alt="privacy icon" width="100">
</center>

## Privacy
<a id="privacy"></a>

In the event that these records contain sensitive <b>personal information</b> (such as medical/employment info), special attention must be paid to preserving this privacy via <i>'de-identification'</i>.

- This means that identifying information are often removed or encrypted before we get them, adding to the challenge of our task.
- This is especially important for academic or medical researchers using [HIPAA](https://en.wikipedia.org/wiki/Health_Insurance_Portability_and_Accountability_Act)-protected datasets for research use.
    - Accountability and oversight are usually much greater in those cases.

<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/41/Noun_project_network_icon_1365244_cc.svg/574px-Noun_project_network_icon_1365244_cc.svg.png" alt="complexity icon" width="150">
</center>


## Computational complexity
<a id="complexity"></a>

As a naive approach, one might try comparing each record in one database, to each record in the other, to determine if each pair under consideration might be a match.

- The computational complexity - $O(N²)$ 
    - The time it takes to use this approach <b>grows <i>quadratically</i></b> with the size of the smaller database.
- This is fine for very small databases (hundreds of records).
    - But, it becomes infeasible when dealing with today's typical databases (>hundreds of thousands of records).
- As we'll see, some nice tricks exist to <i>substantially</i> reduce the size of the problem.

<a id="labels"></a>
## Lack of training labels
In the typical (supervised) machine learning approach, labeled training data is used as feedback by a statistical model during the process of training. 
- In some cases, the there is <i>no training data</i> that tells us if two records correspond to the same individual or not.
    - i.e. <i>'unsupervised'</i>
- This can make the evaluation of the matches produced by a model especially challenging.

Next, we'll describe some <b>solutions</b> to several of these problems.
- I'm going to introduce you to the <b><i>classic record linkage approach</i></b>.

<a id="classic"></a>
<font color='grey'><h1>'Classic' record linkage</h1></font>

---

Although, sometimes the entity to be matched is a business, product, or some other object...

- Most commonly, the records we seek to link refer to a real, live <b>person</b> (shown here).

<br>
<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/6/68/Akha_cropped_hires.JPG" alt="People" width="275">
</center>

<b>Examples:</b>
- Customers in a business database
- Constituents in a government database
- Patients in a hospital database

In general, I will use <i>'record linkage'</i> to refer to the matching of records between databases.
- But, this includes the special case of <b><i>'de-duplication'</i></b>, which simply involves using the same approach to find duplicate records in a <i>single</i> database.

<center>
    <img src="assets/rl_pipeline_figure.png" alt="RL pipeline figure" width="750">
    <i>Typical RL pipeline.</i>
</center>

> #### Let's break it down -- starting with <font color='#CC3333 '><i><b>pre-processing</b></i></font>...

<a id="pre"></a>
<font color='#CC3333 '><h2>Pre-processing</h2></font>

---

Records from different databases often vary wildly in their formatting conventions.
- Notice the <i>extremely-common</i> inconsistencies below...

<br>
<center>
    <img src="assets/record_example_A.png" alt="Example record A" width="950">
</center>
<center>
    <img src="assets/record_example_B.png" alt="Example record B" width="950">
</center>

As a result, it falls to us to ensure that the data we want to compare has been properly <i>cleaned</i> and <i>standardized</i>.
- Any inconsistencies <i>must</i> be resolved for successful linkage.

Although the potential problems that may need to be addressed during pre-processing are too numerous to list -- we can refine the process into <b>three major steps</b>:

<b>1. Removing undesired characters/words</b>
- Non-alphanumeric characters
- In some cases, removing irrelevant words (<i>stop words</i>) is useful.

<b>2. Standardize abbreviations and correct typpos</b>
- Use hash mapping to reduce the variation of equivalent values.

<b>3. Parsing input to create new variables (feature engineering)</b>
- As we'll see, parsing our raw data into it's component elements allows us to model each of them individually, often resulting in a better performing model <i>and</i> a greater ability to make inferences about which variables are most important during classification.   

<br>
<img align="right" src="assets/warning.png" width="200">

> Regardless of the specific pre-processing steps that you perform - <b>don't over-write the original data!</b>
> - Otherwise, there is no guarantee that it can be recovered after being transformed.
> - Later, different pre-processing may be desired.
> - Ideally, new copies of the data are created after each major transformation.

<a id="handling"></a>
### Handling missing values & outliers

#### Missing values
- The <b>removal of rows</b> in RL can result in fewer correctly-linked record pairs, so this option <i>is usually avoided</i>.
    - The exception being rows with so little information they are unlikely to be matched anyway.

- The <b>removal of columns</b> can also be detrimental to the accuracy of our linkage.
    - Even if there are many records missing a value, the records without missing values will still be able to use that information for matching.

#### Outliers
Smoothing or removing noisy values can also hurt the accuracy of our linkage, so <i>it is usually not applied</i>.
- Outliers can still contain useful information.
    - For example, an age of '250' could actually be a typo of '25', which would actually be fairly close in string similarity (e.g. edit distance). 
    - This information would be lost if smoothing were applied.
- Our model won't see them anyway! 
    - Unusual values usually don't make it beyond the comparison step, so they aren't likely to strongly affect our predictions, anyways.

<a id="phonetic"></a>
### Phonetic encoding
As part of standardizing our data, we will often need to convert strings into their [phonetic encoding](https://en.wikipedia.org/wiki/Phonetic_algorithm).

- This allows us to compare words based on the way they are spoken, rather than the way they are spelled.
    - The latter can be more <i>regionally-specific</i>.
- Specifically, we will convert a string (e.g. a name) into a coded representation of how it is spoken.
    - Example codes: Soundex, Phonex, Fuzzy Soundex
- These approaches are linguistic in nature, and so specific to a particular language (usually English).

> After encoding, these de-duplicated features can be useful for <b>'blocking'</b>, which we'll describe in the following section on Indexing.

<a id="segmentation"></a>
### Segmentation
As part of feature engineering, we often break down many of our variables (e.g. full name) into several new variables (e.g. title, first name, last name).
<br>
<center>
    <img src="assets/segmentation.png" alt="Segmentation example" width="1000">
</center>

- This is often the most challenging step in pre-processing, because of all the possible ways to parse our data into new features.
- In general, this will lead to better matching by making more information available during the comparison step.



<b>Rule-based</b> segmentation approaches work well for well-structured fields containing consistent information.
- Examples: regex, `if-then` statements, etc.
- Although, manual approaches like these require a greater invest of time, they can become quite robust with enough labor (and iterations) invested.
    - Unfortunately, they will still always be susceptible to unexpected variations in the data.

<b>Statistical</b> approaches to segmentation work by learning probability distributions regarding how elements in a field should be broken down into its constituent tokens.
- This approach entails classification of each element in a field as a particular token.
- More robust to changes in data formatting over time.

The most popular tool for the statistical approach has been the [<b>hidden Markov model</b>](https://en.wikipedia.org/wiki/Hidden_Markov_model).
- Notice, obtaining/creating data to train HMM is sometimes as much effort as assigning rules.

Ideally, after pre-processing, those same records would look something like this below.

<br>
<center>
    <img src="assets/record_clean.png" alt="Example records" width="1300">
</center>

- Our data now contains all attributes from both databases.
- Content has been standardized.
- Contradicting fields have been corrected.
- Abbreviations have been expanded.

<b>After pre-processing</b>, we should <i>at least</i> understand:
1. the number of unique values for each feature
2. the frequency of those values, and how many missing values each feature contains


> We will rely heavily on this information in the next stage: <font color='#CC3333'><b>indexing</b></font>.

<center>
    <img src="assets/indexing.png" alt="indexing" width="450">
</center>

<a id="blocking"></a>
<font color='#CC3333'><h2>Indexing (blocking)</h2></font>

---

Now, we are ready to compare our records to look for a match.
- But, if we are dealing with typical databases containing, say, a million or more records -- clearly we are not capable of comparing <i>one trillion</i> record pairs in a <i>"reasonable"</i>* time span.

<br></br>
<div style="text-align: right">*Ideally, we're talking minutes to hours, not days or weeks.</div>

<center>
<img src="assets/thinker.png" alt="thinker" width="300">
</center>

Like any good algorithm designer -- we can <i>think</i> about where we can <b>save ourselves from doing work</b>.

- The vast majority of record comparisons will be non-matches.

- Especially so for records that are dis-similar along particular dimensions.

- So, we should work with smaller subsets of data that are very likely to contain matching records.

> <b>For example,</b> while matching record pairs may sometimes contain different 'phone number's, they will almost never contain a different 'gender'. 
> - As a result, we can reduce the complexity of our algorithm substantially by simply comparing only records matching on 'gender'.
>
> <center>
<img src="assets/blocking.png" alt="Blocking example" width="1100">
</center>

<b><i>Blocking</i></b>, is a similar approach to indexing, which relies on a small number of such features to reduce the number of comparisons.

- In the previous example, <i>we would only have two blocks</i> (one for 'male' and one for 'female') for each database -- not ideal.
- 'zip code' and phonetically-encoded 'surname' are <i>much</i> better candidates for use as <i>blocking keys</i>.
- For greater improvements in performance, it is also common to block using <i>multiple</i> variables, in succession.

<img align="right" src="assets/warning.png" width="200">

> <b>Warning!</b>: This approach does, however, sometimes miss certain matches that may, for example, contain a typo in one of the <i>blocking keys</i>.
> - In the example above, the record would not have any chance of being matched in the event that 'gender' changed or was entered incorrectly.
> - This highlights the need for careful selection of blocking criteria.

<a id="keys"></a>
### Defining blocking keys

Given that the comparison step (covered in the next section), is the most computationally-expensive step in the RL process, proper selection of <b>blocking keys can have a <i>major</i> impact on efficiency</b>.

#### Data quality/consistency
One major consideration must be the quality of the values in the fields selected as blocking keys. 
- This refers to both their completeness and their frequency distributions.
- Ideally, the feature selected as the blocking key will be as complete as possible (few errors or missing values).

#### Number vs. size of blocks
The frequency distribution of values in a blocked feature will determine the size of the blocks that are compared.
- A <b>small number of large blocks</b> will result in a more comparisons (more computational expense), but are less likely to miss true matches.
- However, <b>large number of small blocks</b> will result in less computational expense, but are more likely to miss true matches.

For example, blocking by 'gender', as we did in our example earlier, would only result in blocks that were half the size of our original dataset -- not much of an improvement when dealing with quadratic time complexity.
- Higher-cardinality variables, such as 'zip code', often result in blocks of appropriate size.
- Using a variable with too high of cardinality will result in a lower likelihood of our blocks containing true matches.
- Fortunately, blocking keys can be optimized just like any other hyper-parameter.

#### Parallel processing
Blocking is also an opportunity to leverage parallel processing approaches to decrease processing time, when that is applicable.

> After <b>indexing</b> is complete, we are ready to begin the <font color='#CC3333'><i><b>comparison</b></i></font> step.


<center>
    <img src="assets/comparison.png" alt="comparison" width="450">
</center>

<a id="comparison"></a>
<font color='#CC3333'><h2>Comparison</h2></font>

---

Next, the similarity between our candidate pairs is calculated by comparing several record attributes.

- Classification of each record pair into "matches" and "non-matches" can be performed using either a <b><i>deterministic</i></b> (rule-based) or <b><i>probabilistic</i></b> (model-based) approach.
    - We'll focus on using a probabilistic approach, in which comparisons between pairs are broken down into feature vectors summarizing their agreement along multiple dimensions.
- With the <b><i>feature vector</i></b> generated during comparison, we can apply any one of hundreds of different classification models.
    - Models are trained to predict whether records are "matches" or not.

- Comparisons can range from simple numeric comparisons, like the difference between ages of each record, to more complex comparison functions, like 'fuzzy' string matching or distances between addresses. 
- A few examples of such comparisons are shown below.
<center>
<img src="assets/record_comparison.png" alt="Record comparison" width="650">
</center>

The result is a <b>comparison vector</b> for each record pair.
- We used approximate comparisons for strings, [edit distance](https://en.wikipedia.org/wiki/Edit_distance) for numbers, and equivalence for Booleans.
- This <i>feature vector</i> is what we use for classification.
- Using those feature vectors, we can apply any available classification models (sklearn, statsmodels, tensorflow, etc.) to attempt to predict whether those records are "matches" or not.

<a id="strings"></a>
### Strings


#### String comparisons
Strings are usually the most likely type to contain errors and typos, so rather than use exact matching, most RL employ comparison functions that return some indication of similarity on a continuum.
- [Edit distance](https://en.wikipedia.org/wiki/Edit_distance) (i.e. [Levenshtein distance](https://en.wikipedia.org/wiki/Levenshtein_distance)): Smallest number of single character insertions, deletions, and substitutions that are required to convert one string into the other.
- [Hamming distance](https://en.wikipedia.org/wiki/Hamming_distance): Measures the minimum number of substitutions required to change one string into the other.
- [Jaro-Winkler distance](https://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance): Developed at the U.S. Census Bureau -- it counts the number of equivalent characters within half the length of the longer string, and the number of transposes in the sets of common strings.

| Distance | Example   |
|------|------|
| Levenshtein | "kitten" and "sitting" is 3 |
| Hamming | "karolin" and "kathrin" is 3 |
| Jaro–Winkler | "martha","marhta" is 0.944 |

In [ ]:
# textdistance - https://pypi.org/project/textdistance/
# install via: $ pip install textdistance

import textdistance

print('Levenshtein:', textdistance.levenshtein('banana', 'bahama') )

print('Hamming:', textdistance.hamming('banana', 'bahama') )

print('Jaro:', textdistance.jaro_winkler('banana', 'bahama') )

<a id="names"></a>
#### Names
...play a big role in RL.
- Since names are part of language, we know there are usually general rules, but those rules have lots of variability.
- e.g. 'Robert', 'Bob', 'Bobby', 'Rob', 'Roberto'

<i>Culture</i>
- First, middle, family name is an Anglo-Saxon convention -- not representative of <i>most</i> of the world.
- Compound names are common in many other cultures (e.g. 'Santiago Ramón y Cajal')

How names change, or stay the same, can also be very <i>culturally-specific</i>.
- In some places, first and last name are sometimes used in the reverse order. 
- For this reason, it also a good idea to include cross comparisons.
    - For example, including a comparison of first name with last name can catch instances of their reversal.

<a id="numbers"></a>
### Numbers
In addition to strings, RL also sometimes includes numerical comparisons.
- For example, comparing records including age or financial data such as income or expenses.
- As with strings, we would like to be able to calculate an approximate similarity between two values.

<i>Typically</i>, we accomplish this by thresholding their difference (or percentage) relative to some cutoff ($d_{max}$).
- We linearly extrapolate anything below that to between 1.0 (perfect similarity) and 0.0 (total dissimilarity).

#### Examples

- Maximum absolute difference ($d$)
$$
d = |n_1-n_2|\\
sim_{absolute\_difference} = \left\{
    \begin{array}{l}
      1.0-\frac{d}{d_{max}} \hspace{3mm} if d < d_{max} , \\
      0.0  \hspace{22mm}  else
    \end{array}
\right.
$$

- Maximum percentage difference ($pd$)

$$
pd = \frac{|n_1-n_2|}{max(|n_1|,|n_2|)}*100
$$

$$
sim_{percentage\_difference} = \left\{
    \begin{array}{l}
      1.0-(\frac{pd}{pd_{max}}) \hspace{3mm} if pd < pd_{max} , \\
      0.0  \hspace{22mm}  else
    \end{array}
\right.
$$

<a id="time"></a>
### Time

We can treat time data ('date's, 'age's, 'time' comparisons, etc.) as a special case of numerical data and use a similar approach.
- For example, the we can calculate an percentage difference for 'age' ($apd$):

$$
apd = \frac{|d_1-d_2|}{max(d_1,d_2)}*100
$$

- ... and then handle it just as we did above:

$$ 
sim_{apd} = \left\{
    \begin{array}{l}
      1.0-(\frac{apd}{apd_{max}}) \hspace{3mm} if apd < apd_{max} , \\
      0.0  \hspace{22mm}  else
    \end{array}
\right.
$$

<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/13/Rotating_earth_%28huge%29.gif/600px-Rotating_earth_%28huge%29.gif" alt="space" width="300">
</center>


<a id="space"></a>
### Space
As geocoding resources are becoming increasingly popular and available, it's worth mentioning that they can be employed in RL, as well.
- Rather than simply comparing the 'address', we can actually calculate the distance (latitude and longitude) between points on Earth.
- The major bottleneck to this process, is the <i>quality of address information</i> that is available to use for geocoding.
- If any address details are missing or erroneous, then the location data provided may be imprecise or incorrect.

> With our <b>comparisons</b> complete, we can begin the <font color='#CC3333'><b><i>classification</i></b></font> step.

<center>
    <img src="assets/classification.png" alt="classification" width="450">
</center>

<a id="classification"></a>
<h2><font color='#CC3333'>Classification</font></h2>

---

Using our comparison vector, we can now use a [<i>classification model</i>](https://en.wikipedia.org/wiki/Statistical_classification) to classify each record pair as either a <b>"match"</b> or <b>"non-match"</b>.
<center>
    <img src="assets/model.png" alt="Modeling example" width="600">
</center>

> We'll skip most the details of the standard modeling approach, as they are well-described.
> - [This resource](http://faculty.marshall.usc.edu/gareth-james/ISL/) provides an excellent introduction to this topic.

In recent years, a variety of machine learning classifiers have been found useful in record linkage applications. 
- It has been recognized that the algorithm classically associated with record linkage -- i.e. the 'probabilistic' approach -- is equivalent to the [Naive Bayes classifier](https://en.wikipedia.org/wiki/Naive_Bayes_classifier).
    - As a result, it suffers from the same (typically un-true) assumption of <i>feature-independence</i>.

- <i>Higher</i> accuracy can often be achieved using other machine learning techniques, such as a single-layer [perceptron](https://en.wikipedia.org/wiki/Perceptron), which <b><i>does not</i></b> rely on the independence assumption\*.
<div style="text-align: right">*See <a href="http://axon.cs.byu.edu/~randy/pubs/wilson.ijcnn2011.beyondprl.pdf" ><i>Wilson (2011)</i></a></div>

#### <i>As always</i>, the model you choose should depend on the complexity of the data.
- The greater the number of <i>complex field comparisons</i>, with many possible values, the more complex the comparison vector.
- Naturally, a more complex model will be more able to utilize the information in those complex fields, giving it the potential to discriminate more accurately.
    - <i>But</i>, complex models will also have a greater tendency to become <b><i>overfit</i></b>.

> #### Almost there!
> - With the predictions of our classifiers in hand, we are now ready to proceed to the <b>assignment</b> step.

<a id="assignment"></a>
### Assignment
---

Given that this operation is performed on each record pair, <i>independently</i> of all other pairs, a given record may classify as a match for more than one record.
- The simplest way (a greedy algorithm), would result in some assignments being not optimal due to [transitive closure](https://en.wikipedia.org/wiki/Transitive_closure).
- It's a classic [<b><i>assignment problem</i></b>](https://en.wikipedia.org/wiki/Assignment_problem)!
- It's an often-overlooked step in the RL process.

<b>For example,</b> imagine you got the following output from your classifier.
- Records $A$ and $B$ have a match weight of $42.21$.
- But record $A$ has a weight of $39.01$ with $C$, and records $B$ and $C$ have a weight of $44.98$.
- Assuming that the record pair list is sorted, a greedy assignment algorithm will assign record $A$ to record $B$ (because the weight is larger than for $(A,C)$)
- But, then it can not assign record $C$ to record $B$ (even though it is optimal) because record $B$ has already been assigned to $A$.  

| * | A | B | C |
|-|-|-|-|
| <b>A</b> | * | 42.21 | 39.01 | 
| <b>B</b> | 42.21 | * | 44.98 |
| <b>C</b> | 39.01 | 44.98 | * |  

- As a result, the assignment would <b><i>not</i> be optimal</b>.

As a result, <b>one-to-one assignments must be <i>enforced manually</i></b>, through an additional step in our RL pipeline.
- This assignment problem can be solved with special algorithms that find the optimal solution over all possible assignments.
    - For example, the [Hungarian algorithm](https://en.wikipedia.org/wiki/Hungarian_algorithm) is a combinatorial optimization algorithm that solves the assignment problem (see next example).
- But like many similar algorithms, it does so in polynomial time (slow).
- This complexity is acceptable, however, given the reductions in problem size achieved via blocking.

In [ ]:
# sample program computing the lowest cost assignment from a cost matrix
# https://software.clapper.org/munkres/index.html

from munkres import Munkres, print_matrix 
# install via: $ pip install munkres

# matrix representing the costs of each of n workers to perform any of n jobs
matrix = [[5, 9, 1],
          [10, 3, 2],
          [8, 7, 4]]
print_matrix(matrix, msg='matrix:')

# assignment problem is to assign jobs to workers in a way that minimizes the total cost.
# each worker can perform only one job and each job can be assigned to only one worker 

m = Munkres()
indexes = m.compute(matrix)

print('lowest costs:')
total = 0
for row, column in indexes:
    value = matrix[row][column]
    total += value
    print(f'({row}, {column}) -> {value}')
print(f'total cost: {total}')

> After our <b>assignments</b> are complete, we can now begin <font color='#CC3333'><b><i>evaluation</i></b></font> of our results.
<center>
    <img src="assets/evaluation.png" alt="evaluation" width="450">
</center>

<a id="evaluation"></a>
<font color='#CC3333'><h2>Evaluation</h2></font>

---

Of course, we still need to determine how successful the linkage algorithm was, in terms of correctly identified pairs.
- To do so, we would prefer to have access to <b><i>labeled</i></b> data that can be used for validation.
    - This would contain the <i>true match status</i> of all possible matches.
- With this, we can precisely evaluate the accuracy of the fitted classifier.

> <b>As always</b>, it is essential that testing data are different than the training data, otherwise <i>over-fitting will occur</i>!

<center>
    <img src="assets/scale.png" alt="scale" width="150">
</center>

In the event we <i>do</i> have labeled data, we must still contend with a <i>formidable <b>class imbalance</b></i>, between "match" vs "non-match" samples.
- Re-sampling is typically not applicable.
- As a result, appropriate classification metrics must be applied.
    - Accuracy (or misclassification rate), for example, can grow very high by classifying everything as "non-match", making it less meaningful for assessing the quality of a set of linked records.

- Instead, [<i><b>precision</b></i> and <i><b>recall</b></i>](https://en.wikipedia.org/wiki/Precision_and_recall) are used. 
    - These can be combined into the [<i>$F$-measure</i>](https://en.wikipedia.org/wiki/F1_score), which is the harmonic mean of precision and recall.
    - But, the <i>ideal</i> balance between precision and recall will <i>depend on your particular application</i> (medical diagnosis, fraud detection, etc.).

#### Possible matches
In this talk, we'll focus on classifying record pairs as either a "match" or "non-match".
- In fact, a third classification can sometimes be useful -- "possible match".
- These records can then undergo <b>clerical review</b>, and the classified records can then be added to the training data for your model, thus improving your classifier.
- This is usually <i>not</i> feasible in most applications, however.

<a id="demo"></a>
<font color='grey'><h1>Demo</h1></font>

---

For this demo, we'll use the [Python Record Linkage Toolkit](https://recordlinkage.readthedocs.io/en/latest/about.html), a very popular and well-maintained library.
- In addition, it comes with a dataset generator, the [Freely Extensible Biomedical Record Linkage](http://users.cecs.anu.edu.au/~Peter.Christen/Febrl/febrl-0.3/febrldoc-0.3/manual.html) (Febrl) package, which we'll use for a brief demo here.
- We'll use it for <i>pre-processing and indexing</i>, but bring in other tools for <i>classification and evaluation</i>.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# record linkage toolkit
# install via: $ pip install recordlinkage
import recordlinkage as rl
from recordlinkage.datasets import load_febrl4

# metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc

In [ ]:
# load demo data -- simulated biomedical records (labeled 'match' or 'no match')
dfA, dfB, true_links = load_febrl4(return_links=True)

print(dfA.shape)
print(dfB.shape)

In [ ]:
# check out dataframe A
dfA.sample(3)

In [ ]:
# check out dataframe B
dfB.sample(3)

In [ ]:
# indexing step
indexer = rl.Index()
indexer.block('postcode') # use 'postcode' as blocking key

candidate_links = indexer.index(dfA, dfB)

print(candidate_links)

In [ ]:
# comparison step
compare_cl = rl.Compare()

## let's imagine we are working with de-identified data and so no names or birthdates are available
# compare_cl.exact('given_name', 'given_name', label='given_name')
# compare_cl.string('surname', 'surname', method='jarowinkler', label='surname')
# compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')

compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1', label='address_1')
compare_cl.string('address_2', 'address_2', label='address_2')
compare_cl.string('address_1', 'address_2', label='address_swapped')

# generate feature array
features = compare_cl.compute(candidate_links, dfA, dfB)
features.head()

#### Let's view an example "match" pair.

In [ ]:
# randomly choose a record number
record_num = np.random.randint(0, len(true_links))

In [ ]:
# view record from database A
dfA[dfA.index==true_links[record_num][0]]

In [ ]:
# view record from database B
dfB[dfB.index==true_links[record_num][1]]

In [ ]:
# view feature vector from record pair
features[[ind==true_links[record_num] for ind in features.index.to_list()]]

<a id="comparing"></a>
## Comparing classifiers

`recordlinkage` includes a variety of powerful [classifiers](https://recordlinkage.readthedocs.io/en/latest/ref-classifiers.html#), which we are now free to use.

- But, now that we have our labels and training data, we are free to apply <i>any one</i> of a variety of classification tools available.
- So, let's use a handful of standard classification models included in [scikit-learn](https://scikit-learn.org/stable/), which range in complexity.
- We could just as easily apply [statsmodels](https://www.statsmodels.org/) or [tensorflow](https://www.tensorflow.org/), instead.

In [ ]:
# convert labels to Booleans to work with sklearn classifiers (inefficient)
labels = [element in true_links for element in features.index.to_list()]

# Create a training and test set
X_train, X_test, y_train, y_test  = train_test_split(features, labels, random_state=0)

model_predictions = dict()

<a id="supervised"></a>
### Supervised classifiers

In [ ]:
def evaluate_model(model):
    clf = model.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    print(np.array([['tn', 'fp'], ['fn', 'tp']]))
    print(confusion_matrix(y_test, predictions))
    print()
    print(f"F1 score: {f1_score(y_test, predictions, average='macro')}")
    return predictions

#### Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

model_predictions['logreg'] = evaluate_model(LogisticRegression(solver='lbfgs', random_state=0))

#### Naive Bayes classifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model_predictions['nb'] = evaluate_model(MultinomialNB())

#### Support Vector Machine

In [ ]:
from sklearn.svm import SVC

model_predictions['svc'] = evaluate_model(SVC(gamma='auto', random_state=0))

#### Random Forest classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_predictions['rf'] = evaluate_model(RandomForestClassifier(n_estimators=10, random_state=0))

#### Multi-layer Perceptron classifier

In [ ]:
from sklearn.neural_network import MLPClassifier

model_predictions['mlp'] = evaluate_model(MLPClassifier(random_state=0))

<a id="unsupervised"></a>
### Unsupervised classifiers

<b>Unfortunately</b>, obtaining ground-truth data is very often not possible and so other solutions must be applied.

#### K-means clustering

In [ ]:
from sklearn.cluster import KMeans

model_predictions['kmeans'] = evaluate_model(KMeans(n_clusters=2, random_state=0))

#### Expectation/Conditional Maximization Algorithm

In [ ]:
from sklearn.mixture import GaussianMixture

model_predictions['ecm'] = evaluate_model(GaussianMixture(n_components=2, random_state=0))

In [ ]:
# compare model performances
fpr = dict()
tpr = dict()
roc_auc = dict()

models = [
    'logreg',
      'nb',
      'svc',
      'rf',
      'mlp',
      'kmeans', 
      'ecm']

# compute ROC curve and ROC area for each model
for model in models:
    fpr[model], tpr[model], _ = roc_curve(y_test, model_predictions[model])
    roc_auc[model] = auc(fpr[model], tpr[model])

In [ ]:
lw = 2
n_classes = len(model_predictions)

plt.figure(figsize=(6,6))

for key in fpr.keys():
    plt.plot(fpr[key], tpr[key], lw=lw,
             label=f'{key} (auc={roc_auc[key]:0.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.title('receiver operating characteristic')
plt.legend(bbox_to_anchor=(1.05, .7))
plt.show();

<a id="conclusions"></a>
<font color='grey'><h1>Conclusions</h1></font>


---

1. Records on people or businesses is often very 'messy' with inconsistent formatting from one database to the next.
    - This makes finding unique matches unlikely. 
    - Instead, for each record there is a handful of plausible matches, each matching to a varying degree.

2. Typically, pre-processing is necessary in order to normalize undesired variation and ensure consistent formatting.

3. Since the naive approach has quadratic time complexity, reducing the size of the task becomes essential.
    - We do so by beginning with smaller subsets of data that are very likely to contain matching records (i.e. blocking).

4. Due to the expense of hand-labeling training data, we often need to use an unsupervised approach to RL.

5. In spite of our attempts to keep nice, complete databases, people still demand privacy (go figure).
    - As a result, fields that include identifying information are often removed or encrypted, adding to the challenge of our task.

6. Classification of each record pair into "matches" and "non-matches" can be performed using either a deterministic (rule-based) or probabilistic (model-based) approach.
    - Using a probabilistic approach, comparisons between pairs are broken down into feature vectors summarizing their agreement along multiple dimensions.
    - With those feature vectors, we can apply any one of hundreds of different classification models to attempt to predict whether those records are "matches" or not.

<a id="resources"></a>
## Resources
- Data Matching by Peter Christen

<center>
    <a href="https://www.springer.com/gp/book/9783642311635">
        <img src="https://images.springer.com/sgw/books/medium/9783642311635.jpg" width="200" alt="Data matching book">
    </a>
</center>
<br>

<center>
    <h1><i>Thank you,</i></h1>
    <img src="https://pybay.com/site_media/static/new/img/PyBay2020-Transparent.3c44537b6c56.png" width="200" alt="pybay">
</center>

<center>
    <h2><font color='blue'>Big thanks to <i>Grace, Luiz, Karen, et al.</i></font></h2>
    <h3><font color='orange'>Another excellent year!</font></h3>
</center>

> ### <i>HMU in [#talk-data-deduplication-with-python](https://pybay2020.slack.com/archives/C018VEY8DKK)!</i>
